In [116]:
from sqlalchemy import create_engine, func
from sqlalchemy.orm import sessionmaker
import sys
import os
import pandas as pd
from sklearn.preprocessing import OneHotEncoder

sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "../")))
from DB.models import init_db, Circuit, Season, RacingWeekend, Driver, Session, SessionResult, Lap, TyreRaceData, Team, DriverTeamSession, TeamCircuitStats

import numpy as np
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker
from itertools import product

# Initialize database connection
global db_session
engine, db_session = init_db()


def all_drivers_tyre(year, round):
	# Query to get the last 30 race sessions globally prior to the specified year/round
	global_past_races = (
		db_session.query(Session.session_id)
		.join(RacingWeekend, RacingWeekend.racing_weekend_id == Session.weekend_id)
		.filter(
			Session.session_type == "Race",
			# Filter races strictly before the specified year/round
			(RacingWeekend.year < year) |
			((RacingWeekend.year == year) & (RacingWeekend.round < round))
		)
		.order_by(RacingWeekend.year.desc(), RacingWeekend.round.desc())  # Order by most recent first
		.limit(30)  # Limit to the last 30 races
		.all()
	)

	# Extract session IDs from the query result
	global_session_ids = [race.session_id for race in global_past_races]

	# Query tyre data for ALL drivers across the last 30 races globally
	global_tyre_data = (
		db_session.query(TyreRaceData.tyre_type, TyreRaceData.a, TyreRaceData.b, TyreRaceData.c)
		.filter(TyreRaceData.race_id.in_(global_session_ids))
		.all()
	)

	# Group and calculate global averages
	global_tyre_stats = {}
	for tyre_type, a, b, c in global_tyre_data:
		if tyre_type == 4 or tyre_type == 5:
			continue
		if tyre_type not in global_tyre_stats:
			global_tyre_stats[tyre_type] = {"a": [], "b": [], "c": []}

		global_tyre_stats[tyre_type]["a"].append(a)
		global_tyre_stats[tyre_type]["b"].append(b)
		global_tyre_stats[tyre_type]["c"].append(c)

	# Calculate the global averages for each tyre type
	global_averaged_tyre_stats = {}
	for tyre_type, stats in global_tyre_stats.items():
		avg_a = sum(stats["a"]) / len(stats["a"]) if stats["a"] else 0
		avg_b = sum(stats["b"]) / len(stats["b"]) if stats["b"] else 0
		avg_c = sum(stats["c"]) / len(stats["c"]) if stats["c"] else 0

		global_averaged_tyre_stats[tyre_type] = {
			"avg_a": avg_a,
			"avg_b": avg_b,
			"avg_c": avg_c,
		}
	
	return global_averaged_tyre_stats

def get_starting_grid(session_id):
	session_results = db_session.query(SessionResult).filter_by(session_id=session_id).all()

	starting_grid = {}
	for pos in session_results:
		starting_grid[pos.position] = pos.driver_id

	return starting_grid

def get_laps(session_id):
	max_lap = db_session.query(func.max(Lap.lap_num)).filter(Lap.session_id == session_id).scalar()

	# If no laps are found, return 0
	return max_lap if max_lap is not None else 0


def get_driver_basetime(driver_id, team_id, circuit_id, quali_session_id):
	# this does need to be changed, as it does technically include the current race in the average even though that wouldnt be known
	circuit_stats = db_session.query(TeamCircuitStats).filter_by(team_id=team_id, circuit_id=circuit_id).first()

	percent_diff = circuit_stats.quali_to_race_percent_diff
	

	lowest_lap = db_session.query(Lap)\
    .filter(
        Lap.session_id == quali_session_id,  # ensure it's the same session
        Lap.driver_id == driver_id
    )\
    .order_by(Lap.lap_time.asc())\
    .first().lap_time


	return lowest_lap * (1 + percent_diff/100)
	


def get_all_data(year, round):
	# get all drivers that competed
	racing_weekend = db_session.query(RacingWeekend).filter_by(year=year, round=round).first()

	team_circuit_stats = racing_weekend.circuit.team_circuit_stats

	quali_session = db_session.query(Session).filter_by(weekend_id=racing_weekend.racing_weekend_id, session_type="Qualifying").first()
	race_session = db_session.query(Session).filter_by(weekend_id=racing_weekend.racing_weekend_id, session_type="Race").first()

	# find drivers
	drivers = db_session.query(DriverTeamSession).filter_by(session_id=race_session.session_id).all()

	all_driver_tyre_deg = all_drivers_tyre(year, round)
	driver_tyre_deg = {}
	base_laptimes = {}
	for driver_entry in drivers:

		driver = driver_entry.driver

		base_laptime = get_driver_basetime(driver.driver_id, driver_entry.team.team_id, racing_weekend.circut_id, quali_session.session_id)
		base_laptimes[driver.driver_id] = base_laptime

		# Query to get the last 30 race sessions the driver participated in
		past_races = (
			db_session.query(Session.session_id)
			.join(DriverTeamSession, DriverTeamSession.session_id == Session.session_id)
			.join(RacingWeekend, RacingWeekend.racing_weekend_id == Session.weekend_id)
			.filter(
				DriverTeamSession.driver_id == driver.driver_id,
				Session.session_type == "Race",
				# Filter races strictly before the specified year/round
				(RacingWeekend.year < year) |
				((RacingWeekend.year == year) & (RacingWeekend.round < round))
			)
			.order_by(RacingWeekend.year.desc(), RacingWeekend.round.desc())  # Order by most recent first
			.limit(50)  # Limit to the last 30 races
			.all()
		)

		# for race in past_races:
		# 	print(race.round)
		# print("\n\n\n")

		if len(past_races) < 30:
			driver_tyre_deg[driver.driver_id] = all_driver_tyre_deg
			continue

		session_ids = [race.session_id for race in past_races]

		# Get all tyre data for those sessions and driver
		tyre_data = db_session.query(TyreRaceData.tyre_type, TyreRaceData.a, TyreRaceData.b, TyreRaceData.c)\
			.filter(TyreRaceData.driver_id == driver.driver_id,
					TyreRaceData.race_id.in_(session_ids))\
			.all()
		
		# print(len(tyre_data))
		tyre_stats = {}

		# Loop over the queried tyre_data
		for tyre_type, a, b, c in tyre_data:
			if tyre_type == 4 or tyre_type==5:
				continue
			if tyre_type not in tyre_stats:
				tyre_stats[tyre_type] = {"a": [], "b": [], "c": []}
			
			# Append the values of a, b, and c for this tyre type
			tyre_stats[tyre_type]["a"].append(a)
			tyre_stats[tyre_type]["b"].append(b)
			tyre_stats[tyre_type]["c"].append(c)

		# Calculate the averages for each tyre type
		averaged_tyre_stats = {}
		for tyre_type, stats in tyre_stats.items():
			avg_a = sum(stats["a"]) / len(stats["a"]) if stats["a"] else 0
			avg_b = sum(stats["b"]) / len(stats["b"]) if stats["b"] else 0
			avg_c = sum(stats["c"]) / len(stats["c"]) if stats["c"] else 0
			
			averaged_tyre_stats[tyre_type] = {
				"avg_a": avg_a,
				"avg_b": avg_b,
				"avg_c": avg_c,
			}

		driver_tyre_deg[driver.driver_id] = averaged_tyre_stats

	starting_grid = get_starting_grid(quali_session.session_id)
	
	num_laps = get_laps(race_session.session_id)

	return starting_grid, driver_tyre_deg, base_laptimes, num_laps


starting_grid, driver_tyre_deg, base_laptimes, num_laps = get_all_data(2024,4)

Exhaustive search

In [117]:
def simulate_pit_stops(driver_tyre_deg, base_laptimes, num_laps, pit_time=20, max_fuel_kg=110, fuel_effect_per_kg=0.03):
    optimal_pit_stops = {}
    
    for driver_id, tyre_data in driver_tyre_deg.items():
        possible_tyres = list(tyre_data.keys())
        
        # Precompute cumulative lap times for each tyre, including fuel effects
        cumulative = {}
        for tyre in possible_tyres:
            a = tyre_data[tyre]['avg_a']
            b = tyre_data[tyre]['avg_b']
            c = tyre_data[tyre]['avg_c']
            cumulative[tyre] = [0.0]  # index 0 is 0 laps
            total_time = 0.0
            for x in range(1, num_laps + 1):
                # Calculate fuel weight for this lap
                fuel_weight = max_fuel_kg - (x - 1) * (max_fuel_kg / num_laps)
                # Calculate fuel correction for this lap
                fuel_correction = fuel_weight * fuel_effect_per_kg
                # Base lap
                # lap_time = base_laptimes[driver_id]
                # Add tyre
                lap_time = a * (x ** 2) + b * x + c
                # Adjust lap time with fuel correction
                # lap_time += fuel_correction
                # Accumulate total time
                total_time += lap_time
                cumulative[tyre].append(total_time)
                
        # Initialize variables to track the best strategy
        min_total = float('inf')
        best_strategy = None
        
        # Generate all possible strategies with 1 to 4 pit stops
        from itertools import combinations_with_replacement, permutations
        
        for num_pits in range(1, 5):  # 1 to 4 pit stops
            # Generate all possible tyre sequences for the given number of stints
            for tyres in permutations(possible_tyres, num_pits + 1):  # num_pits + 1 stints
                # Ensure at least two different tyre compounds are used
                if len(set(tyres)) < 2:
                    continue
                
                # Generate all possible stint lengths
                from itertools import combinations
                
                # Split laps into `num_pits + 1` stints
                for stint_lengths in combinations(range(1, num_laps), num_pits):
                    stint_lengths = sorted(stint_lengths)
                    stint_lengths = [stint_lengths[0]] + \
                                    [stint_lengths[i] - stint_lengths[i - 1] for i in range(1, len(stint_lengths))] + \
                                    [num_laps - stint_lengths[-1]]
                    
                    # Ensure all stint lengths are valid
                    if any(length <= 0 for length in stint_lengths):
                        continue
                    
                    # Calculate total time for this strategy
                    total_time = 0.0
                    for i, (tyre, length) in enumerate(zip(tyres, stint_lengths)):
                        if length > len(cumulative[tyre]) - 1:
                            break  # Invalid stint length
                        total_time += cumulative[tyre][length]
                        if i > 0:  # Add pit time for all but the first stint
                            total_time += pit_time
                    
                    # Update the best strategy if this one is better
                    if total_time < min_total:
                        min_total = total_time
                        best_strategy = {
                            'pit_laps': [sum(stint_lengths[:i]) for i in range(1, len(stint_lengths))],
                            'tyres_used': tyres,
                            'total_time': total_time
                        }
        
        # If no valid strategy was found, set default values
        if best_strategy is None:
            best_strategy = {
                'pit_laps': [],
                'tyres_used': (),
                'total_time': float('inf')
            }
        
        # Store the best strategy for this driver
        optimal_pit_stops[driver_id] = best_strategy
    
    return optimal_pit_stops

# Example usage:
starting_grid, driver_tyre_deg, base_laptimes, num_laps = get_all_data(2023, 15)
optimal_pits = simulate_pit_stops(driver_tyre_deg, base_laptimes, num_laps)

# Convert the optimal_pits dictionary to a DataFrame
import pandas as pd
df_optimal_pits = pd.DataFrame.from_dict(
    optimal_pits, 
    orient='index', 
    columns=['pit_laps', 'tyres_used', 'total_time']
).reset_index().rename(columns={'index': 'driver_id'})

(df_optimal_pits)

,driver_id,pit_laps,tyres_used,total_time
0,35,[24],"(2, 3)",122.570780
1,17,[1],"(2, -1)",21.444823
2,38,[23],"(2, 3)",114.726809
3,33,[1],"(3, -1)",20.872775
4,1,[1],"(2, -1)",21.543223
5,25,[1],"(3, -1)",20.927057
6,32,[1],"(3, -1)",21.138919
7,4,[1],"(1, -1)",21.271534
8,14,[61],"(-1, 1)",21.058993
9,3,[61],"(-1, 1)",21.415239


In [118]:
from sqlalchemy.orm import Session

def get_driver_names_and_sort_results(df_optimal_pits, session: Session):
    # Step 1: Sort the DataFrame by total_time in ascending order
    df_sorted = df_optimal_pits.sort_values(by='total_time', ascending=True)
    
    # Step 2: Query the database to get driver names
    driver_ids = df_sorted['driver_id'].tolist()  # Extract driver IDs from the sorted DataFrame
    drivers = session.query(Driver).filter(Driver.driver_id.in_(driver_ids)).all()  # Fetch all relevant drivers
    
    # Create a mapping of driver_id to driver_name
    driver_id_to_name = {driver.driver_id: driver.driver_name for driver in drivers}
    
    # Step 3: Add driver_name to the DataFrame
    df_sorted['driver_name'] = df_sorted['driver_id'].map(driver_id_to_name)
    
    # Reorder columns for better readability (optional)
    df_sorted = df_sorted[['driver_id', 'driver_name', 'pit_laps', 'tyres_used', 'total_time']]
    
    # Step 4: Reset the index to start from 1
    df_sorted = df_sorted.reset_index(drop=True)  # Drop the old index
    df_sorted.index += 1  # Start the index from 1 instead of 0
    
    return df_sorted

# Example usage:
# Assuming `session` is your SQLAlchemy session object
sorted_results = get_driver_names_and_sort_results(df_optimal_pits, db_session)

# Display the sorted results with driver names and reindexed rows
sorted_results

,driver_id,driver_name,pit_laps,tyres_used,total_time
1,2,Sergio Perez,[61],"(-1, 1)",20.767902
2,33,Yuki Tsunoda,[1],"(3, -1)",20.872775
3,25,Esteban Ocon,[1],"(3, -1)",20.927057
4,12,Lewis Hamilton,[1],"(3, -1)",20.996645
5,14,Carlos Sainz,[61],"(-1, 1)",21.058993
6,32,Fernando Alonso,[1],"(3, -1)",21.138919
7,4,Lance Stroll,[1],"(1, -1)",21.271534
8,11,Lando Norris,[1],"(2, -1)",21.412197
9,3,Charles Leclerc,[61],"(-1, 1)",21.415239
10,17,Valtteri Bottas,[1],"(2, -1)",21.444823
